In [19]:
import os
import shutil
from tqdm import tqdm
import random

In [20]:
ACTUAL_PATH = os.getcwd()
IMAGES = os.path.join(ACTUAL_PATH, "..\\YOLO_train\\faces\\images\\train")
LABELS = os.path.join(ACTUAL_PATH, "..\\YOLO_train\\faces\\labels\\train")

OUTPUT_PATH = os.path.join(ACTUAL_PATH, "..\\YOLO_train\\datasets")

In [21]:
def change_labels(labels):
    for label in labels:
        label_path = os.path.join(LABELS, label)
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label_data = f.read().strip()

            with open(label_path, 'w') as label_file:
                updated_labels = []
                for line in label_data.splitlines():
                    parts = line.split()  # Découpe la ligne par espaces
                    if len(parts) >= 5:  # Assure qu'il y a au moins la classe et 4 coordonnées
                        parts[0] = str(26)  # Remplace l'indice de classe
                        updated_labels.append(" ".join(parts))  # Reconstruis la ligne
                label_file.write("\n".join(updated_labels) + "\n")

In [22]:
def make_dirs(train_test):
    for type in train_test:
        os.makedirs(os.path.join(ACTUAL_PATH, OUTPUT_PATH, type, "images"), exist_ok=True)
        os.makedirs(os.path.join(ACTUAL_PATH, OUTPUT_PATH, type, "labels"), exist_ok=True)

In [23]:
def split_data(images, labels, train_number, test_number):
    data = list(zip(images, labels))
    random.shuffle(data)
    train_data = data[:train_number]
    test_data = data[train_number:train_number + test_number]
    return train_data, test_data

In [24]:
def move_to_dir(data, train_or_test):

    images_path_destination = os.path.join(ACTUAL_PATH, OUTPUT_PATH, train_or_test, "images")
    labels_path_destination = os.path.join(ACTUAL_PATH, OUTPUT_PATH, train_or_test, "labels")

    data_images = [img for img, _ in data]
    data_labels = [label for _, label in data]

    for img_name in data_images:
        img_path = os.path.join(IMAGES, img_name)
        image_path_dest = os.path.join(images_path_destination, img_name)
        shutil.copy(img_path, image_path_dest)

    for label_name in data_labels:
        label_path = os.path.join(LABELS, label_name)
        label_path_dest = os.path.join(labels_path_destination, label_name)
        shutil.copy(label_path, label_path_dest)


In [25]:
train_test = ["train", "test"]
make_dirs(train_test)

In [26]:
images = os.listdir(IMAGES)
labels = os.listdir(LABELS)
change_labels(labels)
train_data, test_data = split_data(images, labels, 300,30)

In [27]:
move_to_dir(train_data, "train")
move_to_dir(test_data, "test")